In [1]:
import google.generativeai as genai # Aquí está el cambio principal
from google.generativeai import types # Y aquí, para los tipos
import dotenv
import os
import json

dotenv.load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_AI_API_KEY"))

# Tus estructuras de datos no necesitan ser modificadas
tasks_tools_definition = json.loads(open("tools.json", "r").read())
tools = [types.Tool(function_declarations=tasks_tools_definition)]

MODEL_NAME = "gemini-1.5-flash"


def prompt(contents: list) -> list:
    
    model = genai.GenerativeModel(MODEL_NAME)
    
    response = model.generate_content(
        contents=contents,
        tools=tools
    )

    return [
        *contents,
        response.to_dict()['candidates'][0]['content']
    ]

c:\Users\kekew\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
res = prompt([{
    "role": "user",
    "parts": [
        {
            "text": "agrega pan a la lista de compras de la 2da quincena de ju"
        }
    ]
}])

In [5]:
res

[{'role': 'user',
  'parts': [{'text': 'agrega pan a la lista de compras de la 2da quincena de julio'}]},
 {'parts': [{'function_call': {'name': 'despensa.add',
     'args': {'payday': '2024-07-2', 'article': 'pan'},
     'id': ''}}],
  'role': 'model'}]